Text Summarizing

In [4]:
from summarizer import Summarizer
from transformers import logging, AutoConfig, AutoTokenizer, AutoModel
logging.set_verbosity_error()
import warnings
warnings.filterwarnings("ignore")

# alternate: bert-large-uncased (better summary)
custom_config = AutoConfig.from_pretrained('mrm8488/bert-tiny-finetuned-squadv2')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('mrm8488/bert-tiny-finetuned-squadv2')
custom_model = AutoModel.from_pretrained('mrm8488/bert-tiny-finetuned-squadv2', config=custom_config)

model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

In [6]:
f = open("sample.txt","r")
full_text = f.read()
result = model(full_text, min_length=60, max_length = 500 , ratio = 0.4)
summarized_text = ''.join(result)
print (summarized_text)

India has a very ancient tradition of art, which has exchanged many influences with the rest of Eurasia, especially in the first millennium, when Buddhist art spread with Indian religions to Central, East and South-East Asia, the last also greatly influenced by Hindu art.[377] Thousands of seals from the Indus Valley Civilization of the third millennium BCE have been found, usually carved with animals, but a few with human figures. The "Pashupati" seal, excavated in Mohenjo-daro, Pakistan, in 1928–29, is the best known.[378][379] After this there is a long period with virtually nothing surviving.[379][380] Almost all surviving ancient Indian art thereafter is in various forms of religious sculpture in durable materials, or coins.


extract N (best) nouns from text

In [66]:
import nltk
nltk.download('stopwords')

import pke
import string
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chiragmanjeshwar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [68]:
def get_n_nouns(text, n):
  text = text.lower()
  
  extractor = pke.unsupervised.MultipartiteRank()
  stoplist = list(string.punctuation)
  stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
  stoplist += stopwords.words('english')
  extractor.load_document(input=text, stoplist=stoplist)

  pos = {'PROPN'}

  extractor.candidate_selection(pos=pos)
  max_n = len(extractor.candidates)

  if (len(extractor.candidates) < max_n):
    return [i for i in extractor.candidates if i in text]

  for i in range(1, max_n+1):
    extractor.candidate_weighting(alpha=1.1,
                                    threshold=0.75,
                                    method='average')
    nouns = [i[0] for i in extractor.get_n_best(i) if i[0] in text]
    if len(nouns) == n:
      return nouns

  return nouns

In [69]:
get_n_nouns(full_text, 4)

['hindu', 'ellora', 'karla', 'east']